In [1]:
import cv2
import csv
import collections
import numpy as np
from tracker import *
import os
import pandas as pd
# Initialize Tracker
tracker = EuclideanDistTracker()
input_size = 480
# Detection confidence threshold
confThreshold =0.1
nmsThreshold= 0.2

In [2]:
with open("Up_Data.csv", 'a+', newline='') as f1:
    cwriter = csv.writer(f1)
    cwriter.writerow(['Direction','Car', 'Motorbike', 'Bus', 'Truck'])
f1.close()
with open("Down_Data.csv", 'a+', newline='') as f1:
    cwriter = csv.writer(f1)
    cwriter.writerow(['Direction', 'Car', 'Motorbike', 'Bus', 'Truck'])
f1.close()

In [3]:
  # Middle cross line position
middle_line_position = 350
up_line_position = middle_line_position - 10
down_line_position = middle_line_position + 10

In [4]:
font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 1

In [5]:
# Store Coco Names in a list
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)
print(len(classNames))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80


In [6]:
# class index for our required detection classes
required_class_index = [2, 3, 5, 7]

detected_classNames = []

In [7]:
## Model Files
modelConfiguration = 'yolov3-320.cfg'
modelWeights = 'yolov3-320.weights'
# configure the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
# Configure the network backend
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

In [8]:
# Function for finding the center of a rectangle
def find_center(x, y, w, h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy=y+y1
    return cx, cy

In [9]:
# List for store vehicle count information
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]

# Function for count vehicle
def count_vehicle(box_id, img):

    x, y, w, h, id, index = box_id

    # Find the center of the rectangle for detection
    center = find_center(x, y, w, h)
    ix, iy = center
    
    # Find the current position of the vehicle
    if (iy > up_line_position) and (iy < middle_line_position):

        if id not in temp_up_list:
            temp_up_list.append(id)

    elif iy < down_line_position and iy > middle_line_position:
        if id not in temp_down_list:
            temp_down_list.append(id)
            
    elif iy < up_line_position:
        if id in temp_down_list:
            temp_down_list.remove(id)
            up_list[index] = up_list[index]+1

    elif iy > down_line_position:
        if id in temp_up_list:
            temp_up_list.remove(id)
            down_list[index] = down_list[index] + 1

    # Draw circle in the middle of the rectangle
    cv2.circle(img, center, 2, (0, 0, 255), -1)  # end here
#     print(up_list, down_list)

In [10]:
# Function for finding the detected objects from the network output
def postProcess(outputs,img):
    global detected_classNames 
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if classId in required_class_index:
                if confidence > confThreshold:
                    # print(classId)
                    w,h = int(det[2]*width) , int(det[3]*height)
                    x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
                    boxes.append([x,y,w,h])
                    classIds.append(classId)
                    confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
    # print(classIds)
    for i in indices.flatten():
        x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        # print(x,y,w,h)

        color = [int(c) for c in colors[classIds[i]]]
        name = classNames[classIds[i]]
        detected_classNames.append(name)
        # Draw classname and confidence score 
        cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
                  (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Draw bounding rectangle
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        detection.append([x, y, w, h, required_class_index.index(classIds[i])])

    # Update the tracker for each object
    boxes_ids = tracker.update(detection)
    for box_id in boxes_ids:
        count_vehicle(box_id, img)


In [11]:
def realTime():
    try:
        while True:
            success, img = cap.read()
            img = cv2.resize(img,(0,0),None,0.5,0.5)
            ih, iw, channels = img.shape
            blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

            # Set the input of the network
            net.setInput(blob)
            layersNames = net.getLayerNames()
            outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]
            # Feed data to the network
            outputs = net.forward(outputNames)

            # Find the objects from the network output
            postProcess(outputs,img)

            # Draw the crossing lines

            cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 2)
            cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 2)
            cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 2)
    
            # Draw counting texts in the frame
            cv2.putText(img, "Up", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Car:        "+str(up_list[0])+"     "+ str(down_list[0]), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Motorbike:  "+str(up_list[1])+"     "+ str(down_list[1]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Bus:        "+str(up_list[2])+"     "+ str(down_list[2]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Truck:      "+str(up_list[3])+"     "+ str(down_list[3]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

            # Show the frames
            cv2.imshow('Output', img)

            if cv2.waitKey(1) == ord('q'):
                break

        # Write the vehicle counting information in a file and save it
    except Exception as e:
        with open("Up_Data.csv", 'a+', newline='') as f1:
            cwriter = csv.writer(f1)
            up_list.insert(0, "Up")
            cwriter.writerow(up_list)
        f1.close()
        
        with open("Down_Data.csv", 'a+', newline='') as f2:
            cwriter = csv.writer(f2)
            down_list.insert(0, "Down")
            cwriter.writerow(down_list)
        f2.close()
        
        # print("Data saved at 'data.csv'")
        # Finally realese the capture object and destroy all active windows
        cap.release()
        cv2.destroyAllWindows()
        pass

In [15]:
path = 'D:\\Capstone_Project_videos'
files = os.listdir(path)
# print(files)


In [23]:
vid = []
for i in range(5):
    f = files[i:i+5]
print(f)

['VID_20211027_162951.mp4', 'VID_20211027_163005.mp4', 'VID_20211027_163019.mp4', 'VID_20211027_163032.mp4', 'VID_20211027_163045.mp4']


In [12]:
path = 'D:\\Capstone_Project_videos'
files = os.listdir(path)
print(files)

location_list = []
for j in range(1, int(len(files)+1)):
    location_name = 'Location_{}'.format(j)
    location_list.append(location_name)

['VID_20211027_162856.mp4', 'VID_20211027_162910.mp4', 'VID_20211027_162923.mp4', 'VID_20211027_162937.mp4', 'VID_20211027_162951.mp4', 'VID_20211027_163005.mp4', 'VID_20211027_163019.mp4', 'VID_20211027_163032.mp4', 'VID_20211027_163045.mp4', 'VID_20211027_163059.mp4', 'VID_20211027_163111.mp4', 'VID_20211027_163126.mp4', 'VID_20211027_163139.mp4', 'VID_20211027_163157.mp4', 'VID_20211027_163211.mp4', 'VID_20211027_163223.mp4', 'VID_20211027_163247.mp4', 'VID_20211027_163300.mp4', 'VID_20211027_163312.mp4', 'VID_20211027_163324.mp4', 'VID_20211027_163338.mp4', 'VID_20211027_163350.mp4', 'VID_20211027_163403.mp4', 'VID_20211027_163416.mp4', 'VID_20211027_163429.mp4', 'VID_20211027_163442.mp4', 'VID_20211027_163455.mp4', 'VID_20211027_163509.mp4', 'VID_20211027_163522.mp4', 'VID_20211027_163534.mp4', 'VID_20211027_163547.mp4', 'VID_20211027_163600.mp4', 'VID_20211027_163614.mp4', 'VID_20211027_163630.mp4', 'VID_20211027_163644.mp4', 'VID_20211027_163657.mp4', 'VID_20211027_163710.mp4', 

In [ ]:
for i in files:
    i = 'D:\\Capstone_Project_videos\\'+i #addres
    cap = cv2.VideoCapture(i)
    realTime()
    up_list = [0, 0, 0, 0]
    down_list = [0, 0, 0, 0]

In [15]:
location_list

['Location_1',
 'Location_2',
 'Location_3',
 'Location_4',
 'Location_5',
 'Location_6',
 'Location_7',
 'Location_8',
 'Location_9',
 'Location_10',
 'Location_11',
 'Location_12',
 'Location_13',
 'Location_14',
 'Location_15',
 'Location_16',
 'Location_17',
 'Location_18',
 'Location_19',
 'Location_20',
 'Location_21',
 'Location_22',
 'Location_23',
 'Location_24',
 'Location_25',
 'Location_26',
 'Location_27',
 'Location_28',
 'Location_29',
 'Location_30',
 'Location_31',
 'Location_32',
 'Location_33',
 'Location_34',
 'Location_35',
 'Location_36',
 'Location_37',
 'Location_38',
 'Location_39',
 'Location_40',
 'Location_41',
 'Location_42',
 'Location_43',
 'Location_44',
 'Location_45',
 'Location_46',
 'Location_47',
 'Location_48',
 'Location_49',
 'Location_50',
 'Location_51',
 'Location_52',
 'Location_53',
 'Location_54',
 'Location_55',
 'Location_56',
 'Location_57',
 'Location_58',
 'Location_59',
 'Location_60',
 'Location_61',
 'Location_62',
 'Location_63',
 

In [16]:
df_Loc = pd.DataFrame(location_list, columns=['Location'])
df_Loc

,Location
0,Location_1
1,Location_2
2,Location_3
3,Location_4
4,Location_5
...,...
214,Location_215
215,Location_216
216,Location_217
217,Location_218


In [17]:
df_UP = pd.read_csv("Up_Data.csv")
df_UP

,Direction,Car,Motorbike,Bus,Truck
0,Up,21,0,0,6
1,Up,21,0,0,5
2,Up,23,0,0,3
3,Up,18,0,0,7
4,Up,10,0,0,4
5,Up,13,0,0,10
6,Up,22,0,1,5
7,Up,63,0,0,2
8,Up,26,0,0,4
9,Up,65,0,0,5


In [18]:
UP = pd.concat([df_UP, df_Loc], axis=1)
UP.to_csv("UP.csv")

In [19]:
df_DOWN = pd.read_csv("Down_Data.csv")
df_DOWN

,Direction,Car,Motorbike,Bus,Truck
0,Down,10,0,0,1
1,Down,17,0,0,0
2,Down,13,0,0,4
3,Down,10,0,0,1
4,Down,11,0,0,1
5,Down,11,0,0,1
6,Down,8,0,0,6
7,Down,10,0,0,3
8,Down,12,0,0,2
9,Down,15,0,0,2


In [20]:
UP = pd.concat([df_DOWN, df_Loc], axis=1)
UP.to_csv("DOWN.csv")

In [19]:
# for i in files:
#     i = 'C:\\Users\\Raj\\PycharmProjects\\Capstone_Project(Sem3)\\Week_9_Code\\Code and videos\\Vehicle Detection Raw Videos\\'+i #addres
#     cap = cv2.VideoCapture(i)
#     realTime()

In [20]:
# with open("data.csv", 'a+') as f1:
#     cwriter = csv.writer(f1)
#     cwriter.writerow(['Direction', 'car', 'motorbike', 'bus', 'truck'])
#     up_list.insert(0, "Up")
#     down_list.insert(0, "Down")
#     cwriter.writerow(up_list)
#     cwriter.writerow(down_list)
# f1.close()

In [21]:
# print(net.getUnconnectedOutLayers())

In [22]:
# print(len(net.getLayerNames()))

In [23]:
# layersNames = net.getLayerNames()

In [24]:
# layersNames[201]